In [1]:
import codecs
import pandas as pd
import numpy as np
import nltk
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import csv
nltk.download('wordnet')
nltk.download('omw-1.4')
import sklearn
import pymorphy2
from keras import models
from keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RYZEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RYZEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\RYZEN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\RYZEN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
dtf=pd.read_csv('dtf.csv')
dtf

,Unnamed: 0,title,type
0,0,диван вилсон велюр,5
1,1,столик трансформера,6
2,2,рюмка сапожок хрусталь,1
3,3,веник баня берёзовый принять,2
4,4,шкаф многоцелевой назначение оксфорд,4
...,...,...,...
4421,4421,штора школа детский сад ламбрекен,7
4422,4422,стенка гостиный шкаф комод сервант стеллаж,4
4423,4423,поставка антисептик рука поверхность,3
4424,4424,диванкровать юннаорфей,5


In [36]:
#naive bayes
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dtf['title'], dtf['type'], test_size=0.3,random_state=42)
print(x_train)
X_train=pd.DataFrame(x_train)
print(X_train)
X_test=pd.DataFrame(x_test)
freq_train = pd.Series(' '.join(X_train['title']).split()).value_counts()
print(freq_train)
less_five_freq_train = freq_train[(freq_train <10)]
print(less_five_freq_train)

freq_test = pd.Series(' '.join(X_test['title']).split()).value_counts()
less_five_freq_test = freq_test[(freq_test <10)]
#print(less_five_freq_test)


X_train['title']=X_train['title'].apply(lambda x: ' '.join(x for x in x.split() if x not in less_five_freq_train))
X_test['title']= X_test['title'].apply(lambda x: ' '.join(x for x in x.split() if x not in less_five_freq_test))

283       родентицидный средство грызун абсолёный гранула
1298    средство дезинсекция дератизация борьба насеко...
898                             нефрит баня сауна зелёный
3139                                        веник дубовый
3695                     банный принадлежность мебель дом
                              ...                        
3444                                   веник баня наличие
466                      стол журнальный винный дуб соном
3092                         штора летний веранда беседка
3772                            стул лион февраль комфорт
860                   термосящик изотермический контейнер
Name: title, Length: 3098, dtype: object
                                                  title
283     родентицидный средство грызун абсолёный гранула
1298  средство дезинсекция дератизация борьба насеко...
898                           нефрит баня сауна зелёный
3139                                      веник дубовый
3695                   банный принадлежно

In [27]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:


vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=20000)

tfidf_no_stop_training_features = vectorizer.fit_transform(X_train['title'])
tfidf_no_stop_test_features = vectorizer.transform(X_test['title'])

print(tfidf_no_stop_training_features.shape)

(3098, 2080)


In [38]:
from sklearn.metrics import classification_report, confusion_matrix

# A function to plot and print result
def printResult(y_pred, y_prob):
    acc = accuracy_score(y_test, y_pred)
    # Result
    print("Accuracy: {:.2f}".format(acc * 100), end='\n\n')
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', cm)
    print(classification_report(y_test, y_pred))


In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
model = MultinomialNB()
# Training
model.fit(tfidf_no_stop_training_features,y_train)

# Evaluation
y_pred_tfidf_no_stop_nb = model.predict(tfidf_no_stop_test_features)

y_prob_tfidf_no_stop_nb = model.predict_proba(tfidf_no_stop_test_features)[:,1]
print(printResult(y_pred_tfidf_no_stop_nb, y_prob_tfidf_no_stop_nb))

Accuracy: 82.08

Confusion Matrix:
 [[151  21   1   0   0   0   0   5]
 [  3 148   4   1   5   0   1   3]
 [  1  27 141   0   0   0   2   1]
 [  5  59   0  88   1   1   0   3]
 [  1  19   0   1 165   7   2   1]
 [  0  13   0   0   8 142   0   2]
 [  1   7   3   0   5   7  95   1]
 [  3  12   0   0   0   0   1 160]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       178
           1       0.48      0.90      0.63       165
           2       0.95      0.82      0.88       172
           3       0.98      0.56      0.71       157
           4       0.90      0.84      0.87       196
           5       0.90      0.86      0.88       165
           6       0.94      0.80      0.86       119
           7       0.91      0.91      0.91       176

    accuracy                           0.82      1328
   macro avg       0.87      0.82      0.83      1328
weighted avg       0.87      0.82      0.83      1328

None
